### List of all possible icons

https://fontawesome.com/v4/icons/
- BautifyIcon - OG code - https://github.com/python-visualization/folium/blob/main/folium/plugins/beautify_icon.py

In [33]:
import random
import math
import pandas as pd
import folium
import numpy as np
import geopandas 
import requests
import branca

# College Station
# Top Right - 30.757470, -96.249537
# Bottom Left - 30.586657, -96.461866


# Example usage:
min_latitude, max_latitude = 30.586657, 30.757470
# min_latitude, max_latitude = 30.586657, 31.757470
min_longitude, max_longitude = -96.461866, -96.249537


max_dist = 1
crowd_fraction = 0.7

num_points = 50

# Tamu a
tamu_location = (30.612178, -96.341015)
# Ross HEB
heb = (30.616305, -96.314587)
# Northgate
northgate = (30.623859, -96.347556)
# George Bush Library 
lib = (30.596121, -96.354207)
# Bryan
bryan = (30.673732, -96.369490)
# test_location = (30.612178, -96.50)

crowd_centers = [tamu_location, heb, lib, bryan]  # Example crowd centers
# crowd_centers = [tamu_location]


def generate_random_points(min_lat, max_lat, min_lon, max_lon, crowd_centers, n):
    data_points = []

    for _ in range(n):

        if random.random() < crowd_fraction:
            # Select a random crowd center
            center = random.choice(crowd_centers)

            # Generate a random distance and angle
            distance = random.uniform(0, max_dist)  # Adjust the maximum distance as needed
            angle = random.uniform(0, 2 * math.pi)

            # Convert distance and angle to latitude and longitude differentials
            lat_diff = distance * math.cos(angle) / 111.32  # 1 degree of latitude is approximately 111.32 km
            lon_diff = distance * math.sin(angle) / (111.32 * math.cos(center[0]))

            # Calculate the new latitude and longitude
            new_lat = center[0] + lat_diff
            new_lon = center[1] + lon_diff

            # Ensure the generated point is within the specified boundaries            
            new_lat = max(min_lat, min(new_lat, max_lat))
            new_lon = max(min_lon, min(new_lon, max_lon))

        else:
            new_lat = random.uniform(min_lat, max_lat)
            new_lon = random.uniform(min_lon, max_lon)

        data_points.append((new_lat, new_lon))

    return data_points

data_points = generate_random_points(min_latitude, max_latitude, min_longitude, max_longitude, crowd_centers, num_points)

# Create a DataFrame from the generated random points
data_points = pd.DataFrame(data_points, columns=['Latitude', 'Longitude'])

In [2]:
tower_df = pd.read_csv('./data/Cellular_Towers_V1.csv')
filtered_tower_df = tower_df[(tower_df['X']>min_longitude) & (tower_df['X']<max_longitude) & (
    tower_df['Y']>min_latitude) & (tower_df['Y']<max_latitude) ]

In [50]:
truckdata = {
    'Longitude': [-96.465841],
    'Latitude': [30.629284]
}
truck_points = pd.DataFrame(truckdata)

In [57]:
from folium.plugins import BeautifyIcon
from folium.plugins import MousePosition
from folium.plugins import HeatMap

# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6,tiles="OpenStreetMap",control_scale =True)

lgd_txt = '<span style="color: {col};">Crowd</span>'
color = 'red'
# Add markers for each data point
data = folium.FeatureGroup(lgd_txt.format( txt= color+' egg', col= "#055C9D") ).add_to(m)
for index, row in data_points.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], opacity=0.7,
                  icon = BeautifyIcon(
                prefix = "fa", icon="user", border_color="blue", text_color="blue" #,  icon_size=(5,5)
                  )).add_to(data)

lgd_txt = '<span style="color: {col};">Towers</span>'
tower = folium.FeatureGroup(lgd_txt.format( txt= color+' egg', col= "#922B21")).add_to(m)
for ind, row in filtered_tower_df.iterrows():
    folium.Marker(location = [row['latdec'], row['londec']],icon = BeautifyIcon(
    prefix = "fa", icon="signal", border_color="#922B21", text_color="#922B21", text_size = 100,  icon_size=(50,50), inner_icon_style="font-size:24px;padding-top:5px;",
)).add_to(tower)
    
lgd_txt = '<span style="color: {col};">Truck</span>'
truck = folium.FeatureGroup(lgd_txt.format( txt= color+' egg', col= "#4CBB17")).add_to(m)
for index, row in truck_points.iterrows():
    folium.Marker(location = [row['Latitude'], row['Longitude']],icon = BeautifyIcon(
    icon="truck", border_color="#4CBB17", text_color="#4CBB17", icon_shape=None, icon_size=(60,60), inner_icon_style="font-size:35px;padding-top:5px;",
)).add_to(truck)

# Create a HeatMap layer using population data
lgd_txt = '<span style="color: {col};">Crowd Heatmap</span>'
heat_data = [[row['Latitude'], row['Longitude']] for index, row in data_points.iterrows()]
HeatMap(heat_data, min_opacity=0.1, blur = 30, name= lgd_txt.format( txt= color+' egg', col= "#055C9D") ).add_to(m)

m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)
folium.map.LayerControl('topleft', collapsed= False).add_to(m)

def fit_bounds(points, m):
    sw = points[['Latitude', 'Longitude']].min().values.tolist()
    ne = points[['Latitude', 'Longitude']].max().values.tolist()
    m.fit_bounds([sw, ne])

fit_bounds(data_points, m)
m

In [15]:
from folium.plugins import HeatMap
# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6)



from folium.plugins import BeautifyIcon
from folium.plugins import MousePosition

# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6,tiles="OpenStreetMap",control_scale =True)

tower = folium.FeatureGroup().add_to(m)
for ind, row in filtered_tower_df.iterrows():
    folium.Marker(location = [row['latdec'], row['londec']],icon = BeautifyIcon(
    # prefix = "fa", icon="tower", border_color="#b3334f", text_color="#b3334f"
    # prefix = "fa", icon="broadcast-tower", border_color="#b3334f", text_color="#b3334f"
    prefix = "fa", icon="signal", border_color="#b3334f", text_color="#b3334f",  icon_size=(00,00)
)).add_to(tower)
    
truck = folium.FeatureGroup().add_to(m)
for index, row in truck_points.iterrows():
    folium.Marker(location = [row['Latitude'], row['Longitude']],icon = BeautifyIcon(
    icon="truck", border_color="#b3334f", text_color="#b3334f", icon_shape="triangle"
)).add_to(truck)

folium.TileLayer("Stamen Watercolor").add_to(m)
folium.TileLayer("CartoDB Positron", show=False).add_to(m)

# Create a HeatMap layer using population data
heat_data = [[row['Latitude'], row['Longitude']] for index, row in data_points.iterrows()]
HeatMap(heat_data, min_opacity=0.1, blur = 30).add_to(m)

m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)

def fit_bounds(points, m):
    # df = pd.DataFrame(points).rename(columns={0:'Lat', 1:'Lon'})[['Lat', 'Lon']] #update
    sw = points[['Latitude', 'Longitude']].min().values.tolist()
    ne = points[['Latitude', 'Longitude']].max().values.tolist()
    m.fit_bounds([sw, ne])

fit_bounds(data_points, m)
# m

In [64]:
data = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()
states = geopandas.GeoDataFrame.from_features(data, crs="EPSG:4326")

states['strength'] = np.random.randint(0, 101, size=len(states))
states.head()

,geometry,name,strength
0,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",Alabama,19
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",Alaska,57
2,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",Arizona,41
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas,1
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",California,37


In [65]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import random

# Define colormap for signal strength
colormap = branca.colormap.LinearColormap(
    vmin=states["strength"].quantile(0.0),
    vmax=states["strength"].quantile(1),
    colors=["red", "orange", "lightblue", "green", "darkgreen"],
    caption="Signal Strength",
)
# Create a Folium map centered on the USA
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

popup = folium.GeoJsonPopup(
    fields=["name", "strength"],
    aliases=["State", "% Change"],
    localize=True,
    labels=True,
    style="background-color: yellow;",
)

# Create a tooltip
tooltip = folium.GeoJsonTooltip(
    fields=["State", "strength"],
    aliases=["State:", "strength:"],
    localize=True,
    sticky=False,
    labels=True,
    style="font-weight: bold;",
    max_width=800,
)

# Create a GeoDataFrame with the 'State', 'Geometry', and 'Signal_Strength' columns
data = {'State': states['name'], 'geometry': states['geometry'], 'strength': states['strength']}
gdf = gpd.GeoDataFrame(data, geometry=states['geometry'], crs="EPSG:4326")


# Create a GeoJSON layer with style and tooltip
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["strength"])
        if x["properties"]["strength"] is not None else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
).add_to(m)

# Add colormap to the map
colormap.add_to(m)

# Save the map as an HTML file
m.save('usa_map.html')
m

In [66]:
tower_df = pd.read_csv('./data/Cellular_Towers_V1.csv')

In [67]:
tower_df.head()

,X,Y,OBJECTID,UniqSysID,Licensee,Callsign,LocNum,LatDeg,LatMin,LatSec,...,Nepa,QZone,TowReg,SupStruc,AllStruc,StrucType,LicStatus,latdec,londec,url
0,-81.284778,38.637083,1,11497,AT&T Mobility Spectrum LLC,KNKN848,6,38,38,13.5,...,N,,1002593,123.1,125.6,GTOWER,A,38.637083,-81.284778,http://wireless2.fcc.gov/UlsApp/UlsSearch/lice...
1,-81.618028,39.037417,2,11497,AT&T Mobility Spectrum LLC,KNKN848,7,39,2,14.7,...,N,,0,55.2,59.1,LTOWER,A,39.037417,-81.618028,http://wireless2.fcc.gov/UlsApp/UlsSearch/lice...
2,-81.082000,38.959389,3,11497,AT&T Mobility Spectrum LLC,KNKN848,18,38,57,33.8,...,N,,1274613,121.9,128.0,LTOWER,A,38.959389,-81.082000,http://wireless2.fcc.gov/UlsApp/UlsSearch/lice...
3,-81.625000,38.665278,4,11497,AT&T Mobility Spectrum LLC,KNKN848,9,38,39,55.0,...,N,,1034261,123.1,125.9,GTOWER,A,38.665278,-81.625000,http://wireless2.fcc.gov/UlsApp/UlsSearch/lice...
4,-81.915000,38.848444,5,11497,AT&T Mobility Spectrum LLC,KNKN848,10,38,50,54.4,...,N,,1219729,80.8,83.8,LTOWER,A,38.848444,-81.915000,http://wireless2.fcc.gov/UlsApp/UlsSearch/lice...


In [68]:
from folium.plugins import BeautifyIcon

In [69]:
from folium.plugins import BeautifyIcon

# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6,tiles="OpenStreetMap",control_scale =True)

# Add markers for each data point
data = folium.FeatureGroup().add_to(m)
for index, row in data_points.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(data)


truck = folium.FeatureGroup().add_to(m)
for index, row in truck_points.iterrows():
    folium.Marker(location = [row['Latitude'], row['Longitude']],icon = BeautifyIcon(
    icon="truck", border_color="#b3334f", text_color="#b3334f", icon_shape="triangle"
)).add_to(truck)
    

tower = folium.FeatureGroup().add_to(m)
for index, row in tower_df.sample(100).iterrows():
    folium.Marker(location = [row['latdec'], row['londec']],icon = BeautifyIcon(
    icon="tower-observation", border_color="#b3334f", text_color="#b3334f", icon_shape="triangle"
)).add_to(tower)
    

folium.TileLayer("Stamen Watercolor").add_to(m)
folium.TileLayer("CartoDB Positron", show=False).add_to(m)

#folium.LatLngPopup().add_to(m)
m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)

# Display the map
m.save('map.html')  # Save the map to an HTML file
m


In [12]:
# from streamlit_folium import st_folium

# st_data = st_folium(m, width=725)

2023-10-09 09:39:32.325 
  command:

    streamlit run d:\Softwares\Installed\Anaconda\envs\move_it\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [10]:
# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6,tiles="OpenStreetMap",control_scale =True)

# Add markers for each data point
data = folium.FeatureGroup().add_to(m)
for index, row in data_points.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']]).add_to(data)

truck = folium.FeatureGroup().add_to(m)
for index, row in truck_points.iterrows():
    folium.Marker(location = [row['Latitude'], row['Longitude']],icon = BeautifyIcon(
    icon="truck", border_color="#b3334f", text_color="#b3334f", icon_shape="triangle"
)).add_to(truck)
    
signal= folium.FeatureGroup().add_to(m)
# Add colormap to the map
# Create a GeoJSON layer with style and tooltip
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["strength"])
        if x["properties"]["strength"] is not None else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
).add_to(signal)
colormap.add_to(m)

folium.TileLayer("Stamen Watercolor").add_to(m)
folium.TileLayer("CartoDB Positron", show=False).add_to(m)

folium.LatLngPopup().add_to(m)
m.fit_bounds(m.get_bounds())
folium.LayerControl().add_to(m)

# Display the map
m.save('map.html')  # Save the map to an HTML file
m


AttributeError: module 'folium' has no attribute 'plugins'

In [11]:
from folium.plugins import HeatMap
# Create a map centered around the first location in your data
m = folium.Map(location=[data_points['Latitude'].iloc[0], data_points['Longitude'].iloc[0]], zoom_start=6)

# Create a HeatMap layer using population data
heat_data = [[row['Latitude'], row['Longitude']] for index, row in data_points.iterrows()]
HeatMap(heat_data).add_to(m)

# Display the map
m.save('population_heatmap.html')  # Save the map to an HTML file
m